# Performance considerations

In [10]:
from time import clock
from timeit import timeit
import memory_profiler
%load_ext memory_profiler

import numpy as np
from scipy.linalg import eigh
from scipy.spatial.distance import squareform, pdist

import pandas as pd
from matplotlib.pyplot import plot, scatter
import seaborn as sns
%matplotlib inline

from sklearn.datasets import make_blobs
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import adjusted_mutual_info_score, accuracy_score, v_measure_score, normalized_mutual_info_score


In [19]:
#!/usr/bin/env python3

"""
COPAC: Correlation Partition Clustering
"""

# Author: Roman Feldbauer <roman.feldbauer@ofai.at>
#         ... <>
#         ... <>
#         ... <>
#
# License: ...
from multiprocessing import cpu_count

import numpy as np
from scipy import linalg as LA
from scipy.spatial.distance import squareform

from sklearn.base import BaseEstimator, ClusterMixin
from sklearn.cluster.dbscan_ import dbscan
from sklearn.neighbors import NearestNeighbors
from sklearn.utils import check_array


def _cdist(P, Q, Mhat_P):
    """ Correlation distance between P and Q (not symmetric).

    Notes
    -----
    The squareroot of cdist is taken later. The advantage here is to
    save some computation, as we can first take the maximum of
    two cdists, and then take the root of the 'winner' only.
    """
    PQ_diff = P[np.newaxis, :] - Q
    return (PQ_diff @ Mhat_P * PQ_diff).sum(axis=1)


def copac(X, k=10, mu=5, eps=0.5, alpha=0.85, metric='euclidean',
          metric_params=None, algorithm='auto', leaf_size=30, p=None,
          n_jobs=1, sample_weight=None):
    """Perform COPAC clustering from vector array.
    Read more in the :ref:`User Guide <copac>`.

    Parameters
    ----------
    X : array of shape (n_samples, n_features)
        A feature array.
    k : int, optional, default=10
        Size of local neighborhood for local correlation dimensionality.
        The paper suggests k >= 3 * n_features.
    mu : int, optional, default=5
        Minimum number of points in a cluster with mu <= k.
    eps : float, optional, default=0.5
        Neighborhood predicate, so that neighbors are closer than `eps`.
    alpha : float in ]0,1[, optional, default=0.85
        Threshold of how much variance needs to be explained by Eigenvalues.
        Assumed to be robust in range 0.8 <= alpha <= 0.9 [see Ref.]
    metric : string, or callable
        The metric to use when calculating distance between instances in a
        feature array. If metric is a string or callable, it must be one of
        the options allowed by sklearn.metrics.pairwise.pairwise_distances
        for its metric parameter.
        If metric is "precomputed", `X` is assumed to be a distance matrix and
        must be square.
    metric_params : dict, optional
        Additional keyword arguments for the metric function.
    algorithm : {'auto', 'ball_tree', 'kd_tree', 'brute'}, optional
        The algorithm to be used by the scikit-learn NearestNeighbors module
        to compute pointwise distances and find nearest neighbors.
        See NearestNeighbors module documentation for details.
    leaf_size : int, optional (default = 30)
        Leaf size passed to BallTree or cKDTree. This can affect the speed
        of the construction and query, as well as the memory required
        to store the tree. The optimal value depends
        on the nature of the problem.
    p : float, optional
        The power of the Minkowski metric to be used to calculate distance
        between points.
    n_jobs : int, optional, default=1
        Number of parallel processes. Use all cores with n_jobs=-1.
    sample_weight : None
        Currently ignored

    Returns
    -------
    labels : array [n_samples]
        Cluster labels for each point. Noisy samples are given the label -1.

    References
    ----------
    Elke Achtert, Christian Bohm, Hans-Peter Kriegel, Peer Kroger,
    A. Z. (n.d.). Robust, complete, and efficient correlation
    clustering. In Proceedings of the Seventh SIAM International
    Conference on Data Mining, April 26-28, 2007, Minneapolis,
    Minnesota, USA (2007), pp. 413–418.
    """
    t0 = clock()
    t_start = t0
    X = check_array(X)
    n, d = X.shape
    y = -np.ones(n, dtype=np.int)
    if n_jobs == -1:
        n_jobs = cpu_count()
    t1 = clock()
    print(f"init took {t1 - t0:.3f} sec.")
    t0 = clock()
    # Calculating M^ just once requires lots of memory...
    lambda_ = np.zeros(n, dtype=int)
    M_hat = list()

    # Get nearest neighbors
    nn = NearestNeighbors(n_neighbors=k, metric=metric, algorithm=algorithm,
                          leaf_size=leaf_size, metric_params=metric_params,
                          p=p, n_jobs=n_jobs)
    nn.fit(X)
    knns = nn.kneighbors(return_distance=False)
    t1 = clock()
    print(f"k-NN took {t1 - t0:.3f} sec.")
    t0 = clock()
    for P, knn in enumerate(knns):
        N_P = X[knn]

        # Corr. cluster cov. matrix
        Sigma = np.cov(N_P[:, :], rowvar=False, ddof=0)

        # Decompose spsd matrix, and sort Eigenvalues descending
        E, V = LA.eigh(Sigma)
        del Sigma
        E = np.sort(E)[::-1]

        # Local correlation dimension
        explanation_portion = np.cumsum(E) / E.sum()
        lambda_P = np.searchsorted(explanation_portion, alpha, side='left')
        lambda_P += 1
        lambda_[P] = lambda_P
        # Correlation distance matrix
        E_hat = (np.arange(1, d + 1) > lambda_P).astype(int)
        M_hat.append(V @ np.diag(E_hat) @ V.T)
    t1 = clock()
    print(f"corr dim took {t1 - t0:.3f} sec.")
    t0 = clock()
    # Group pts by corr. dim.
    argsorted = np.argsort(lambda_)
    edges, _ = np.histogram(lambda_[argsorted], bins=np.arange(1, d + 2))
    Ds = np.split(argsorted, np.cumsum(edges))
    # Loop over partitions according to local corr. dim.
    max_label = 0
    used_y = np.zeros_like(y, dtype=bool)
    t1 = clock()
    print(f"grouping took {t1 - t0:.3f} sec.")
    t0 = clock()
    for D in Ds:
        n_D = D.shape[0]
        cdist_P = -np.ones(n_D * (n_D - 1) // 2, dtype=np.float)
        cdist_Q = -np.ones((n_D, n_D), dtype=np.float)
        start = 0
        t1 = clock()
        print(f"init took {t1 - t0:.3f} sec.")
        t0 = clock()
        # Calculate triu part of distance matrix
        for i in range(0, n_D - 1):
            p = D[i]
            # Vectorized inner loop
            q = D[i + 1:n_D]
            stop = start + n_D - i - 1
            cdist_P[start:stop] = _cdist(X[p], X[q], M_hat[p])
            start = stop
        t1 = clock()
        print(f"triu took {t1 - t0:.3f} sec.")
        t0 = clock()
        # Calculate tril part of distance matrix
        for i in range(1, n_D):
            q = D[i]
            p = D[0:i]
            cdist_Q[i, :i] = _cdist(X[q], X[p], M_hat[q])
        # Extract tril to 1D array
        # TODO simplify...
        cdist_Q = cdist_Q.T[np.triu_indices_from(cdist_Q, k=1)]
        t1 = clock()
        print(f"tril took {t1 - t0:.3f} sec.")
        t0 = clock()
        cdist = np.block([[cdist_P], [cdist_Q]])
        # Square root of the higher value of cdist_P, cdist_Q
        cdist = np.sqrt(cdist.max(axis=0))
        t1 = clock()
        print(f"... took {t1 - t0:.3f} sec.")
        t0 = clock()
        # Perform DBSCAN with full distance matrix
        cdist = squareform(cdist)
        clust = dbscan(X=cdist, eps=eps, min_samples=mu,
                       metric='precomputed', n_jobs=n_jobs)
        _, labels = clust
        t1 = clock()
        print(f"dbscan took {t1 - t0:.3f} sec.")
        t0 = clock()
        # Each DBSCAN run is unaware of previous ones,
        # so we need to keep track of previous cluster IDs
        y_D = labels + max_label
        new_labels = np.unique(labels[labels >= 0]).size
        max_label += new_labels
        # TODO check correct indexing of label array `y`
        y[D] = y_D
        used_y[D] = True
        t1 = clock()
        print(f"rest took {t1 - t0:.3f} sec.")
        t0 = clock()
    t1 = clock()
    print(f"corr dist matrices took {t1 - t0:.3f} sec.")
    t0 = clock()
    assert np.all(used_y), "Not all samples got labels!"
    t_stop = t0
    print(f"Total: {t_stop - t_start:.3f} sec.")
    return y


class COPAC(BaseEstimator, ClusterMixin):
    """Perform COPAC clustering from vector array.
    Read more in the :ref:`User Guide <copac>`.

    Parameters
    ----------
    k : int, optional, default=10
        Size of local neighborhood for local correlation dimensionality.
        The paper suggests k >= 3 * n_features.
    mu : int, optional, default=5
        Minimum number of points in a cluster with mu <= k.
    eps : float, optional, default=0.5
        Neighborhood predicate, so that neighbors are closer than `eps`.
    alpha : float in ]0,1[, optional, default=0.85
        Threshold of how much variance needs to be explained by Eigenvalues.
        Assumed to be robust in range 0.8 <= alpha <= 0.9 [see Ref.]
    metric : string, or callable
        The metric to use when calculating distance between instances in a
        feature array. If metric is a string or callable, it must be one of
        the options allowed by sklearn.metrics.pairwise.pairwise_distances
        for its metric parameter.
        If metric is "precomputed", `X` is assumed to be a distance matrix and
        must be square.
    metric_params : dict, optional
        Additional keyword arguments for the metric function.
    algorithm : {'auto', 'ball_tree', 'kd_tree', 'brute'}, optional
        The algorithm to be used by the scikit-learn NearestNeighbors module
        to compute pointwise distances and find nearest neighbors.
        See NearestNeighbors module documentation for details.
    leaf_size : int, optional (default = 30)
        Leaf size passed to BallTree or cKDTree. This can affect the speed
        of the construction and query, as well as the memory required
        to store the tree. The optimal value depends
        on the nature of the problem.
    p : float, optional
        The power of the Minkowski metric to be used to calculate distance
        between points.
    n_jobs : int, optional, default=1
        Number of parallel processes. Use all cores with n_jobs=-1.

    Attributes
    ----------
    labels_ : array, shape = [n_samples]
        Cluster labels for each point in the dataset given to fit().
        Noisy samples are given the label -1.

    Notes
    -----
    ...
    References
    ----------
    Elke Achtert, Christian Bohm, Hans-Peter Kriegel, Peer Kroger,
    A. Z. (n.d.). Robust, complete, and efficient correlation
    clustering. In Proceedings of the Seventh SIAM International
    Conference on Data Mining, April 26-28, 2007, Minneapolis,
    Minnesota, USA (2007), pp. 413–418.
    """

    def __init__(self, k=10, mu=5, eps=0.5, alpha=0.85,
                 metric='euclidean', metric_params=None, algorithm='auto',
                 leaf_size=30, p=None, n_jobs=1):
        self.k = k
        self.mu = mu
        self.eps = eps
        self.alpha = alpha
        self.metric = metric
        self.metric_params = metric_params
        self.algorithm = algorithm
        self.leaf_size = leaf_size
        self.p = p
        self.n_jobs = n_jobs

    def fit(self, X, y=None, sample_weight=None):  # @UnusedVariable pylint: disable=unused-argument
        """Perform COPAC clustering from features.

        Parameters
        ----------
        X : ndarray of shape (n_samples, n_features)
            A feature array.
        sample_weight : array, shape (n_samples,), optional
            Weight of each sample, such that a sample with a weight of at least
            ``min_samples`` is by itself a core sample; a sample with negative
            weight may inhibit its eps-neighbor from being core.
            Note that weights are absolute, and default to 1.
        y : Ignored
        """
        X = check_array(X)
        clust = copac(X, sample_weight=sample_weight,
                      **self.get_params())
        self.labels_ = clust  #pylint: disable=attribute-defined-outside-init
        return self

    def fit_predict(self, X, y=None, sample_weight=None):  #pylint: disable=arguments-differ
        """Performs clustering on X and returns cluster labels.

        Parameters
        ----------
        X : ndarray matrix of shape (n_samples, n_features)
            A feature array.
        sample_weight : array, shape (n_samples,), optional
            Weight of each sample, such that a sample with a weight of at least
            ``min_samples`` is by itself a core sample; a sample with negative
            weight may inhibit its eps-neighbor from being core.
            Note that weights are absolute, and default to 1.
        y : Ignored

        Returns
        -------
        y : ndarray, shape (n_samples,)
            cluster labels
        """
        self.fit(X, sample_weight=sample_weight)
        return self.labels_


## Timeit

In [8]:
clstr = COPAC()
for n in [20, 100, 1000, 10000]:
    X = np.random.rand(n, 50)
    %timeit clstr.fit_predict(X)

21.9 ms ± 5.49 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
48.7 ms ± 1.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
658 ms ± 31.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
42.1 s ± 3.85 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Memit

In [12]:
clstr = COPAC()
for n in range(1, 11):
    n = 1000 * n
    X = np.random.rand(n, 50)
    %memit clstr.fit_predict(X)

peak memory: 347.40 MiB, increment: 0.00 MiB
peak memory: 347.40 MiB, increment: 0.00 MiB
peak memory: 447.65 MiB, increment: 100.25 MiB
peak memory: 636.50 MiB, increment: 289.10 MiB
peak memory: 787.20 MiB, increment: 439.80 MiB
peak memory: 1155.91 MiB, increment: 802.97 MiB
peak memory: 1416.86 MiB, increment: 1063.93 MiB
peak memory: 1725.25 MiB, increment: 1372.31 MiB
peak memory: 2141.61 MiB, increment: 1788.67 MiB
peak memory: 2574.90 MiB, increment: 2221.97 MiB


## Profiling

Runtime of (5000, 20) sized matrix: 15 sec.
* initial k-NN: 1 sec.
* calculate correlation dimension: 4 sec.
* correlation distance triu: 4.4 sec.
* correlation distance tril: 4.6 sec.
* correlation distance rest: 0.7 sec. (incl. DBSCAN)

In [20]:
X = np.random.rand(5000, 20)
clstr.fit_predict(X)

init took 0.000 sec.
k-NN took 0.957 sec.
corr dim took 4.134 sec.
grouping took 0.001 sec.
init took 0.000 sec.
triu took 0.000 sec.
tril took 0.000 sec.
... took 0.000 sec.
dbscan took 0.001 sec.
rest took 0.000 sec.
init took 0.000 sec.
triu took 0.000 sec.
tril took 0.000 sec.
... took 0.000 sec.
dbscan took 0.001 sec.
rest took 0.000 sec.
init took 0.000 sec.
triu took 0.000 sec.
tril took 0.000 sec.
... took 0.000 sec.
dbscan took 0.001 sec.
rest took 0.000 sec.
init took 0.000 sec.
triu took 0.000 sec.
tril took 0.000 sec.
... took 0.000 sec.
dbscan took 0.001 sec.
rest took 0.000 sec.
init took 0.002 sec.
triu took 0.076 sec.
tril took 0.082 sec.
... took 0.005 sec.
dbscan took 0.008 sec.
rest took 0.000 sec.
init took 0.249 sec.
triu took 4.363 sec.
tril took 4.568 sec.
... took 0.225 sec.
dbscan took 0.154 sec.
rest took 0.000 sec.
init took 0.003 sec.
triu took 0.002 sec.
tril took 0.001 sec.
... took 0.008 sec.
dbscan took 0.001 sec.
rest took 0.000 sec.
init took 0.000 sec

array([-1, -1, -1, ..., -1, -1, -1])

Runtime of (10000, 20) sized matrix: 49 sec.
* initial k-NN: 4 sec.
* calculate correlation dimension: 10 sec.
* correlation distance triu: 14.6 sec.
* correlation distance tril: 18.0 sec.
* correlation distance rest: 1.6 sec. (incl. DBSCAN)

In [21]:
X = np.random.rand(10000, 20)
clstr.fit_predict(X)

init took 0.000 sec.
k-NN took 3.784 sec.
corr dim took 9.646 sec.
grouping took 0.002 sec.
init took 0.000 sec.
triu took 0.000 sec.
tril took 0.000 sec.
... took 0.000 sec.
dbscan took 0.001 sec.
rest took 0.000 sec.
init took 0.000 sec.
triu took 0.000 sec.
tril took 0.000 sec.
... took 0.000 sec.
dbscan took 0.001 sec.
rest took 0.000 sec.
init took 0.000 sec.
triu took 0.000 sec.
tril took 0.000 sec.
... took 0.000 sec.
dbscan took 0.001 sec.
rest took 0.000 sec.
init took 0.000 sec.
triu took 0.000 sec.
tril took 0.000 sec.
... took 0.000 sec.
dbscan took 0.001 sec.
rest took 0.000 sec.
init took 0.011 sec.
triu took 0.356 sec.
tril took 0.382 sec.
... took 0.042 sec.
dbscan took 0.029 sec.
rest took 0.000 sec.
init took 0.789 sec.
triu took 14.611 sec.
tril took 17.959 sec.
... took 0.875 sec.
dbscan took 0.628 sec.
rest took 0.000 sec.
init took 0.028 sec.
triu took 0.014 sec.
tril took 0.009 sec.
... took 0.096 sec.
dbscan took 0.003 sec.
rest took 0.000 sec.
init took 0.000 s

array([-1, -1, -1, ..., -1, -1, -1])

Reasonable performance. Most time spent in correlation distance, corr. dimension, k-NN